In [1]:
import pandas as pd

In [2]:
df=pd.read_csv(r"D:\uni\AdvancedTopics\Assignment3\DB\card_games\cards.csv")
df=df.sample(20,random_state=42)

C:\Users\Elia\AppData\Local\Temp\ipykernel_22012\3117138291.py:1: DtypeWarning: Columns (11,15,18,39,50) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(r"D:\uni\AdvancedTopics\Assignment3\DB\card_games\cards.csv")


In [3]:
df_rule=pd.read_csv(r"D:\uni\AdvancedTopics\Assignment3\DB\card_games\ruling.csv")

In [4]:
result = df_rule[df_rule['uuid'].isin(df['uuid'])]

In [8]:
df.to_csv("DB/card_games/cards_sample20.csv")
result.to_csv("DB/card_games/ruling_sample.csv")

id      34
date    34
text    34
uuid    34
dtype: int64

In [8]:
from groq import Groq
import pandas as pd
import re
from typing import List, Dict, Union, Optional,Any
import json

# Initialize Groq client
client = Groq(api_key="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")

# ====================== GENERIC UDFs ======================

def analyze_sentiment(text: str) -> float:
    """Sentiment Analysis: Returns score between -1 (negative) and 1 (positive)"""
    if not text:
        return 0.0
    
    chat_completion = client.chat.completions.create(
        messages=[{
            "role": "system",
            "content": "Analyze sentiment of this text. Return ONLY a float between -1 (negative) and 1 (positive)."
        }, {
            "role": "user",
            "content": text
        }],
        model="llama3-70b-8192", 
        temperature=0.0,
        max_tokens=10
    )
    return float(chat_completion.choices[0].message.content)

def align_to_schema(data: Dict, domain: str) -> List[Dict]:
    prompt = f"""Convert this table column mapping to Schema.org/{domain} property names:
    For each column name in the input data, provide the most appropriate Schema.org property name.
    The input data is: {data}
    
    Return ONLY a valid JSON array where each element is an object with:
    - the original column name as key
    - the corresponding Schema.org property name as value
    
    Example output format:
    {{
      "name": "name",
      "flavorText": "description",
      "text": "text",
      "type": "additionalType"
    }}"""
    
    chat_completion = client.chat.completions.create(
        messages=[{
            "role": "user", 
            "content": prompt + "\n\nJSON output required:"
        }],
        model="llama3-70b-8192",
        temperature=0.0,
        response_format={"type": "json_object"}
    )
    
    # Get the response and convert it to a list of mappings
    result = json.loads(chat_completion.choices[0].message.content)
    return result

def classify_entity(text: str, classes: List[str]) -> str:
    """Entity Classification: Categorizes text into predefined classes"""
    prompt = f"""Classify this text into one of {classes}:
    {text}
    Return ONLY the class name."""
    
    chat_completion = client.chat.completions.create(
        messages=[{"role": "user", "content": prompt}],
        model="llama3-70b-8192",
        temperature=0.0,
        max_tokens=20
    )
    return chat_completion.choices[0].message.content.strip()

def summarize(text: str, max_length: int = 50) -> str:
    """Summarization: Condenses text to specified length"""
    prompt = f"""Summarize this in under {max_length} characters:
    {text}
    Summary:"""
    
    chat_completion = client.chat.completions.create(
        messages=[{"role": "user", "content": prompt}],
        model="llama3-70b-8192",
        temperature=0.0,
        max_tokens=max_length
    )
    return chat_completion.choices[0].message.content.strip()

def extract_entities(text: str, entity_types: List[str]) -> List[str]:
    prompt = f"""Extract ALL {entity_types} from this text:
    "{text}"
    Return ONLY a Python-style list like ["item1", "item2"] with no other text."""
    
    chat_completion = client.chat.completions.create(
        messages=[{"role": "user", "content": prompt}],
        model="llama3-70b-8192",
        temperature=0.0,
        max_tokens=100
    )
    
    try:
        return eval(chat_completion.choices[0].message.content)
    except:
        return []
    try:
        # Handle both direct string response and JSON object
        response = chat_completion.choices[0].message.content
        if isinstance(response, str):
            return eval(response)
        elif isinstance(response, dict):
            return list(response.values())[0]  # Assumes first value is the list
    except:
        return []  # Fallback to empty list on parsing failure

def impute_missing(
    row: Dict[str, Any],
    column_to_impute: str,
    context_columns: List[str],
    type_hint: Optional[str] = None,
    id_columns: Optional[List[str]] = None
) -> Dict[str, Any]:
    """
    Generic missing value imputation using specified context columns.
    
    Args:
        row: Input data row as dictionary
        column_to_impute: Key containing missing value
        context_columns: List of columns to use for context
        type_hint: Expected data type ("numeric", "date", "boolean", "text")
        id_columns: Columns to exclude from context (even if in context_columns)
    
    Returns:
        Dictionary with imputed value (or original if imputation fails)
    """
    # Return early if no imputation needed
    if row.get(column_to_impute) not in [None, "NULL", "", "null"] and not pd.isna(row.get(column_to_impute)):
        return row[column_to_impute]
    
    # Filter context columns
    id_columns = id_columns or []
    available_context = [
        (k, str(row[k])) for k in context_columns 
        if k in row and k != column_to_impute 
        and row[k] is not None and k not in id_columns
    ]
    
    # Build context string
    context_str = "\n".join(f"{k}: {v}" for k, v in available_context)
    if not context_str:
        return row[column_to_impute]  # No context available
    
    # Determine type hint
    type_instruction = ""
    if type_hint:
        type_map = {
            "numeric": "Return only a number without symbols",
            "date": "Use YYYY-MM-DD format",
            "boolean": "Return 'true' or 'false'",
            "text": "Return a string without quotes"
        }
        type_instruction = f"\nIMPORTANT: {type_map.get(type_hint, '')}"
    
    prompt = f"""Impute missing value for '{column_to_impute}' based on this context:
    {context_str}
    {type_instruction}
    Return ONLY the raw value with no additional text or formatting."""
    chat_completion = client.chat.completions.create(
        messages=[{"role": "user", "content": prompt}],
        model="llama3-70b-8192",
        temperature=0.0,
        max_tokens=50
    )
    return chat_completion.choices[0].message.content.strip()


def augment_schema(row: Dict, new_field: str, prompt: str) -> Dict:
    """Schema Augmentation: Adds new derived field"""
    user_prompt = f"""{prompt}
    Data: {row}
    Return ONLY the value for {new_field}."""
    
    chat_completion = client.chat.completions.create(
        messages=[{"role": "user", "content": user_prompt}],
        model="llama3-70b-8192",
        temperature=0.0,
        max_tokens=50
    )
    row[new_field] = chat_completion.choices[0].message.content.strip()
    return row


In [3]:
# Load sample data
cards_df = pd.read_csv("DB/card_games/cards_sample20.csv")
rulings_df = pd.read_csv("DB/card_games/ruling_sample.csv")

In [4]:
# Create the list of 10 JSON-style examples as dictionaries for export
examples = [
    {
        "unique_id": "sentiment_analysis_001",
        "database_id": "mtg_db",
        "table_schema": "CREATE TABLE cards (uuid TEXT PRIMARY KEY, name TEXT, flavorText TEXT, text TEXT, type TEXT)",
        "question": "Which cards have flavor text with negative sentiment?",
        "hypothetical_relevant_data": [
            {
                "uuid": "fb2e5522-233a-547f-b600-b81701a2a29e",
                "name": "Twilight Prophet",
                "flavorText": "The blood of the ancients flows in her veins, and the whispers of the dead echo in her mind.",
                "text": "Flying. Ascend (If you control ten or more permanents, you get the city’s blessing for the rest of the game.)...",
                "type": "Creature — Vampire Cleric"
            }
        ],
        "expected_result": ["Twilight Prophet"],
        "udf_justification": "Standard SQL cannot analyze sentiment in text. An LLM UDF is needed to evaluate the emotional tone of flavorText."
    },
    {
        "unique_id": "entity_recognition_001",
        "database_id": "mtg_db",
        "table_schema": "CREATE TABLE cards (uuid TEXT PRIMARY KEY, name TEXT, flavorText TEXT, text TEXT, type TEXT)",
        "question": "Which cards mention creatures or people in their flavor text?",
        "hypothetical_relevant_data": [
            {
                "uuid": "4d40e2b6-9c14-574e-9096-a7974e7c03ca",
                "name": "Dream Stalker",
                "flavorText": "What happens when it is the dream that wakes and the sleeper that fades?",
                "text": "When Dream Stalker enters the battlefield, return a permanent you control to its owner's hand.",
                "type": "Creature — Illusion"
            },
            {
                "uuid": "f5266746-23a5-56f6-8f8b-a317956203ca",
                "name": "Civic Wayfinder",
                "flavorText": "\"These alleys are not safe. Come, I can guide you back to the market square.\"",
                "text": "When Civic Wayfinder enters the battlefield, you may search your library for a basic land card...",
                "type": "Creature — Elf Druid Warrior"
            }
        ],
        "expected_result": ["Civic Wayfinder"],
        "udf_justification": "Identifying mentions of entities like people or creature types in natural language requires named entity recognition."
    },
    {
        "unique_id": "data_cleaning_001",
        "database_id": "mtg_db",
        "table_schema": "CREATE TABLE cards (uuid TEXT PRIMARY KEY, name TEXT, flavorText TEXT, text TEXT, type TEXT)",
        "question": "Find cards where the flavor text appears to contain malformed quotation marks or HTML encoding.",
        "hypothetical_relevant_data": [
            {
                "uuid": "2f908c10-fb4e-5d2a-a007-e7bc9a1a3d86",
                "name": "Helm of the Gods",
                "flavorText": "Its blessings are as numerous as the stars of Nyx&mdash;and just as distant.",
                "text": "Equipped creature gets +1/+1 for each enchantment you control.",
                "type": "Artifact — Equipment"
            }
        ],
        "expected_result": ["Helm of the Gods"],
        "udf_justification": "Standard SQL cannot interpret or clean up malformed HTML entities or character encodings in text fields."
    },
    {
        "unique_id": "schema_alignment_001",
        "database_id": "mtg_db",
        "table_schema": "CREATE TABLE cards (uuid TEXT PRIMARY KEY, name TEXT, flavorText TEXT, text TEXT, type TEXT)",
        "question": "Map the card properties to Schema.org types and properties.",
        "hypothetical_relevant_data": [
            {
                "uuid": "6bded0a1-5a83-5f68-b252-681e5de7ce1e",
                "name": "Lady Sun",
                "flavorText": "Sister to Sun Quan and wife to Liu Bei, Lady Sun often felt torn between family loyalty and personal duty.",
                "text": "{T}: Return Lady Sun and another target creature to their owners' hands.",
                "type": "Legendary Creature — Human Advisor"
            }
        ],
        "expected_result": [
            {
                "name": "Lady Sun",
                "mapped_properties": {
                    "name": "schema:name",
                    "flavorText": "schema:description",
                    "text": "schema:comment",
                    "type": "schema:additionalType"
                }
            }
        ],
        "udf_justification": "Schema alignment to external ontologies like Schema.org requires semantic reasoning beyond SQL."
    },
    {
        "unique_id": "cell_imputation_001",
        "database_id": "mtg_db",
        "table_schema": "CREATE TABLE cards (uuid TEXT PRIMARY KEY, name TEXT, flavorText TEXT, text TEXT, type TEXT)",
        "question": "Fill in the missing flavor text based on other card fields.",
        "hypothetical_relevant_data": [
            {
                "uuid": "8c8d0389-4822-5f04-bd49-b6f58def95ad",
                "name": "Ancestral Memories",
                "flavorText": None,
                "text": "Look at the top seven cards of your library. Put two of them into your hand and the rest into your graveyard.",
                "type": "Sorcery"
            }
        ],
        "expected_result": [
            {
                "uuid": "8c8d0389-4822-5f04-bd49-b6f58def95ad",
                "imputed_flavorText": "\"The thoughts of those who came before still linger like echoes in the library of minds.\""
            }
        ],
        "udf_justification": "Imputing missing creative text requires understanding context and generating content—beyond SQL’s scope."
    },
    {
        "unique_id": "summarization_001",
        "database_id": "mtg_db",
        "table_schema": "CREATE TABLE cards (uuid TEXT PRIMARY KEY, name TEXT, flavorText TEXT, text TEXT, type TEXT); CREATE TABLE rulings (uuid TEXT, date TEXT, ruling TEXT)",
        "question": "Summarize the rulings for the card 'Esper Sojourners'.",
        "hypothetical_relevant_data": [
            {
            "uuid": "ceedc6a5-a4e3-59c4-884d-d85076d8616d",
            "name": "Esper Sojourners",
            "flavorText": None,
            "text": "When you cycle Esper Sojourners or it dies, you may tap target permanent.",
            "type": "Artifact Creature — Vedalken Wizard"
            },
            {
            "uuid": "ceedc6a5-a4e3-59c4-884d-d85076d8616d",
            "date": "2008-10-01",
            "ruling": "Cycling is an activated ability. Effects that prevent activated abilities won't prevent cycling."
            },
            {
            "uuid": "ceedc6a5-a4e3-59c4-884d-d85076d8616d",
            "date": "2009-05-01",
            "ruling": "The triggered ability acts as a cycle-triggered ability."
            }
        ],
        "expected_result": [
            {
            "name": "Esper Sojourners",
            "summary": "Cycling is an activated ability not affected by suppression, and it triggers another ability when used."
            }
        ],
        "udf_justification": "Summarizing multiple rulings into a coherent statement requires semantic aggregation beyond SQL capabilities."
    },
    {
        "unique_id": "multi_udf_001",
        "database_id": "mtg_db",
        "table_schema": "CREATE TABLE cards (uuid TEXT PRIMARY KEY, name TEXT, flavorText TEXT, text TEXT, type TEXT); CREATE TABLE rulings (uuid TEXT, date TEXT, ruling TEXT)",
        "question": "Which cards with negative flavor text sentiment also have complex rulings? Summarize the rulings.",
        "hypothetical_relevant_data": [
            {
            "uuid": "fb2e5522-233a-547f-b600-b81701a2a29e",
            "name": "Twilight Prophet",
            "flavorText": "The blood of the ancients flows in her veins, and the whispers of the dead echo in her mind.",
            "text": "Flying\nAscend (If you control ten or more permanents, you get the city’s blessing for the rest of the game.)...",
            "type": "Creature — Vampire Cleric"
            },
            {
            "uuid": "fb2e5522-233a-547f-b600-b81701a2a29e",
            "date": "2018-01-19",
            "ruling": "If you don’t have the city’s blessing, Twilight Prophet’s ability doesn’t trigger at all."
            },
            {
            "uuid": "fb2e5522-233a-547f-b600-b81701a2a29e",
            "date": "2018-01-19",
            "ruling": "Twilight Prophet’s ability checks whether you have the city’s blessing only as it triggers."
            }
        ],
        "expected_result": [
            {
            "name": "Twilight Prophet",
            "summary": "Twilight Prophet only triggers if you already have the city’s blessing at the time of triggering. Timing is critical.",
            "sentiment": "negative"
            }
        ],
        "udf_justification": "Sentiment analysis is needed for evaluating emotional tone; summarization is needed to condense multiple rulings into one coherent explanation."
    },
    {
        "unique_id": "multi_udf_002",
        "database_id": "mtg_db",
        "table_schema": "CREATE TABLE cards (uuid TEXT PRIMARY KEY, name TEXT, flavorText TEXT, text TEXT, type TEXT)",
        "question": "Classify all Legendary creatures and align their fields to Schema.org.",
        "hypothetical_relevant_data": [
            {
            "uuid": "6bded0a1-5a83-5f68-b252-681e5de7ce1e",
            "name": "Lady Sun",
            "flavorText": "Sister to Sun Quan and wife to Liu Bei, Lady Sun often felt torn between family loyalty and personal duty.",
            "text": "{T}: Return Lady Sun and another target creature to their owners' hands.",
            "type": "Legendary Creature — Human Advisor"
            }
        ],
        "expected_result": [
            {
            "name": "Lady Sun",
            "entity_class": "Legendary Creature",
            "schema_mapping": {
                "name": "schema:name",
                "flavorText": "schema:description",
                "text": "schema:comment",
                "type": "schema:additionalType"
            }
            }
        ],
        "udf_justification": "Requires both classification of the entity type (Legendary Creature) and alignment to an external ontology (Schema.org)."
    },
    {
        "unique_id": "group_by_001",
        "database_id": "mtg_db",
        "table_schema": "CREATE TABLE cards (uuid TEXT PRIMARY KEY, name TEXT, flavorText TEXT, text TEXT, type TEXT)",
        "question": "How many cards of each type have flavor text that includes a quotation?",
        "hypothetical_relevant_data": [
            {
            "uuid": "f5266746-23a5-56f6-8f8b-a317956203ca",
            "name": "Civic Wayfinder",
            "flavorText": "\"These alleys are not safe. Come, I can guide you back to the market square.\"",
            "text": "...",
            "type": "Creature — Elf Druid Warrior"
            },
            {
            "uuid": "f87ae87b-26a7-5b93-86bb-f07c7ec3b091",
            "name": "Bitter Ordeal",
            "flavorText": "\"The world is a meat grinder, and we're all on the menu.\" —Geth",
            "text": "...",
            "type": "Sorcery"
            }
        ],
        "expected_result": [
            {"type": "Creature — Elf Druid Warrior", "count": 1},
            {"type": "Sorcery", "count": 1}
        ],
        "udf_justification": "Quotation detection requires language modeling; GROUP BY aggregation combines counts per card type."
    },
    {
        "unique_id": "join_001",
        "database_id": "mtg_db",
        "table_schema": "CREATE TABLE cards (uuid TEXT PRIMARY KEY, name TEXT, flavorText TEXT, text TEXT, type TEXT); CREATE TABLE rulings (uuid TEXT, date TEXT, ruling TEXT)",
        "question": "Which cards have rulings that mention 'damage prevention'?",
        "hypothetical_relevant_data": [
            {
            "uuid": "609cd717-5e92-5b4d-b04c-2e26b9965d2b",
            "name": "Divine Deflection",
            "flavorText": "Only the divine may dictate the course of fate.",
            "text": "Prevent the next X damage that would be dealt to any number of target creatures and/or players this turn...",
            "type": "Instant"
            },
            {
            "uuid": "609cd717-5e92-5b4d-b04c-2e26b9965d2b",
            "date": "2012-05-01",
            "ruling": "Damage prevention effects apply even if a creature would deal more damage than is prevented."
            }
        ],
        "expected_result": ["Divine Deflection"],
        "udf_justification": "Requires keyword detection in text fields after joining rulings and cards on UUID."
    }
]




In [ ]:
# Save to JSON file
import json

#output_path = "udf_examples_part1.json"
with open(output_path, "w") as f:
    json.dump(examples, f, indent=2)

output_path

'udf_examples_part1.json'

In [ ]:
def test_sentiment_analysis_001(cards: pd.DataFrame) -> List[str]:
    """
    Calculates the sentiment score on cards' flavourtext and returns the negative ones
    "Which cards have flavor text with negative sentiment?"
    Test Case ID: entity_classification_001
    """
    negative_cards = []
    for _, card in cards.iterrows():
        if pd.notna(card['flavorText']):
            sentiment = analyze_sentiment(card['flavorText'])
            if sentiment < 0:  # Negative sentiment
                negative_cards.append(card['name'])
    return negative_cards

print(test_sentiment_analysis_001(cards_df))

['Ostiary Thrull']


In [3]:
def test_entity_classification_001(cards: pd.DataFrame) -> List[Dict]:
    """
    Classifies the archetype of 'Twilight Prophet' based on text using
    "What is the archetype of Twilight Prophet?"
    Test Case ID: entity_classification_001
    """
    # Get the specific card data
    card = cards[cards['name'] == 'Twilight Prophet'].iloc[0]
    archetype=classify_entity(card['text'],["Aggro","Control","Combo"])
    return archetype

print(test_entity_classification_001(cards_df))

Control


In [ ]:
def test_entity_recognition_001(cards: pd.DataFrame) -> List[str]:
    """
    Identifies people or creature entities in a text and returns all the cards that present identified entities in their flavour text
    Which cards mention creatures or people in their flavor text?
    Test Case ID: entity_recognition_001
    """
    entity_cards = []
    for _, card in cards.iterrows():
        if pd.notna(card['flavorText']):
            entities = extract_entities(card['flavorText'], ["PERSON","CREATURE"])
            if entities:  # Found mentions of people/creatures
                entity_cards.append(card['name'])
    return entity_cards

print(test_entity_recognition_001(cards_df))

['Crested Sunmare', 'Civic Wayfinder', 'Helm of the Gods', 'Ancestral Memories', 'Lady Sun']


In [ ]:
def test_schema_alignment_001(cards: pd.DataFrame) -> List[Dict]:
    """
    Map card properties to Schema.org types.
    Test Case ID: schema_alignment_001    
    """
    aligned = []
    for _, card in cards.iterrows():
        if card['name'] == "Lady Sun":  # From test case
            mapped = align_to_schema({
                "name": card['name'],
                "flavorText": card['flavorText'],
                "text": card['text'],
                "type": card['type']
            }, "Game")
            aligned.append({
                "name": card['name'],
                "mapped_properties": mapped
            })
    return aligned
print(test_schema_alignment_001(cards_df))


[{'name': 'Lady Sun', 'mapped_properties': {'name': 'name', 'flavorText': 'description', 'text': 'gameplayDescription', 'type': 'gameItemVariant'}}]


In [ ]:
def test_cell_imputation_001(cards: pd.DataFrame) -> List[Dict]:
    """
    Calls the llm to try and fill a specific column value based on external knowledge and the other columns values 
    "Fill in the missing toughness value of creatures based on other card fields."
    Test Case ID: cell_imputation_001
    """
    # Get the creature type cards
    creature_cards = cards_df[cards_df['type'].str.contains('Creature', case=False, na=False)].copy()
    # List of cards to modify 
    cards_to_update = ["Civic Wayfinder", "Dream Stalker", "Esper Sojourners"]

    # Update toughness to None/NULL for those specific cards
    creature_cards.loc[creature_cards['name'].isin(cards_to_update), 'toughness'] = None
    imputed_cards=[]
    creature_cards=creature_cards.to_dict('records')
    for card in creature_cards:
        if pd.isna(card["toughness"]) or card["toughness"] in [None, "NULL", "", "null"]:
            imputed_cards.append({"name":card["name"],"imputed_toughness":impute_missing(
        row=card,
        column_to_impute="toughness",
        context_columns=["name", "type", "power", "text"],
        type_hint="numeric")})
    return(imputed_cards)
    
print(test_cell_imputation_001(cards_df))

#card = cards[cards['name'] == 'Civic Wayfinder'].iloc[0].to_dict()
#    card["toughness"]=None
    
    # Impute using relevant context fields
#    imputed_toughness = impute_missing(
#        row=card,
#        column_to_impute="toughness",
#        context_columns=["name", "type", "power", "text"],
#        type_hint="numeric",
        #id_columns=["uuid"]
#    )
    
#    return [{
#        "name": card['name'],
#        "imputed_toughness": imputed_toughness # Ensure string output
#    }]

#print(test_cell_imputation_001(cards_df))

[{'name': 'Esper Sojourners', 'imputed_toughness': '2'}, {'name': 'Dream Stalker', 'imputed_toughness': '2'}, {'name': 'Civic Wayfinder', 'imputed_toughness': '3'}]


'card = cards[cards[\'name\'] == \'Civic Wayfinder\'].iloc[0].to_dict()\n    card["toughness"]=None\n\n    # Impute using relevant context fields\n    imputed_toughness = impute_missing(\n        row=card,\n        column_to_impute="toughness",\n        context_columns=["name", "type", "power", "text"],\n        type_hint="numeric",\n        #id_columns=["uuid"]\n    )\n\n    return [{\n        "name": card[\'name\'],\n        "imputed_toughness": imputed_toughness # Ensure string output\n    }]\n\nprint(test_cell_imputation_001(cards_df))'

In [ ]:
def test_summarization_001(cards: pd.DataFrame, rulings: pd.DataFrame) -> List[Dict]:
    """
    Calls the llm to summarize a text
    "Summarize rulings for 'Esper Sojourners'."
    Test Case ID: summarization_001
    """
    summaries = []
    card_name = "Esper Sojourners"
    card_uuid = cards[cards['name'] == card_name]['uuid'].iloc[0]
    card_rulings = rulings[rulings['uuid'] == card_uuid]
    
    if not card_rulings.empty:
        combined_rulings = "\n".join(card_rulings['text'])
        summary = summarize(combined_rulings, 100)
        summaries.append({
            "name": card_name,
            "summary": summary
        })
    return summaries

print(test_summarization_001(cards_df,rulings_df))

[{'name': 'Esper Sojourners', 'summary': 'Cycling is an activated ability, interacts with ability effects, not spell effects.'}]


In [ ]:
def test_join_001(cards: pd.DataFrame, rulings: pd.DataFrame) -> List[str]:
    """
    Identifies cards with rulings classified as "damage_prevention" 
    "Which cards have rulings that mention 'damage prevention'?"
    Test Case ID: join_001
    """
    damage_prevention_cards = []
    
    for _, ruling in rulings.iterrows():
        # Classify the ruling text
        classification = classify_entity(
            text=ruling['text'],
            classes=["damage_prevention", "other"],
            #task_description="""Classify Magic rulings as "damage_prevention" if they:
            #- Mention preventing/avoiding/shielding from damage
            #- Reference damage prevention effects
            #- State damage can/cannot be prevented
            #Otherwise classify as "other"."""
        )
        
        if classification == "damage_prevention":
            card = cards[cards['uuid'] == ruling['uuid']].iloc[0]
            damage_prevention_cards.append(card['name'])
    
    return list(set(damage_prevention_cards))
print(test_join_001(cards_df,rulings_df))

['Twilight Prophet', 'Lady Sun', 'Esper Sojourners', 'Illusions of Grandeur', 'Helm of the Gods', 'Crested Sunmare', 'Blizzard Brawl']


In [ ]:
"""AGGIONRANRE ESEMPIO JSON, hypotetical data e expected result"""
def test_group_by_001(cards: pd.DataFrame) -> List[Dict]:
    """
    call llm to analyze the text to identify cards that give a buff and count how many identified cards for each type
    How many cards of each type have text that inplies giving a buff?
    Test Case ID: join_001
    """
    results = []
    
    # Group by card type and process each group
    for card_type, group in cards.groupby('type'):
        buff_count = 0
        
        for text in group['text'].dropna():  # Skip NaN values
            extracted = extract_entities(text, ["buff", "stat increase"])
            if extracted:
                buff_count += 1
                
        
        results.append({
            "type": card_type,
            "buff_count": buff_count,
            "total_cards": len(group)
        })
    return results

print(test_group_by_001(cards_df))

[{'type': 'Artifact Creature — Horse', 'buff_count': 1, 'total_cards': 1}, {'type': 'Artifact Creature — Vedalken Wizard', 'buff_count': 0, 'total_cards': 1}, {'type': 'Artifact — Equipment', 'buff_count': 1, 'total_cards': 1}, {'type': 'Creature — Elf Druid Warrior', 'buff_count': 0, 'total_cards': 1}, {'type': 'Creature — Horse', 'buff_count': 1, 'total_cards': 1}, {'type': 'Creature — Illusion', 'buff_count': 0, 'total_cards': 1}, {'type': 'Creature — Merfolk Soldier', 'buff_count': 0, 'total_cards': 1}, {'type': 'Creature — Thrull', 'buff_count': 0, 'total_cards': 1}, {'type': 'Creature — Vampire Cleric', 'buff_count': 0, 'total_cards': 1}, {'type': 'Enchantment', 'buff_count': 2, 'total_cards': 2}, {'type': 'Instant', 'buff_count': 1, 'total_cards': 2}, {'type': 'Land', 'buff_count': 1, 'total_cards': 3}, {'type': 'Legendary Creature — Human Advisor', 'buff_count': 0, 'total_cards': 1}, {'type': 'Snow Sorcery', 'buff_count': 1, 'total_cards': 1}, {'type': 'Sorcery', 'buff_count': 

In [46]:
def test_multi_udf_001(cards: pd.DataFrame, rulings: pd.DataFrame) -> List[Dict]:
    """Cards with positive flavor text sentiment and complex rulings."""
    complex_cards = []
    for _, card in cards.iterrows():
        if pd.notna(card['flavorText']):
            sentiment = analyze_sentiment(card['flavorText'])
            if sentiment > 0:
                card_rulings = rulings[rulings['uuid'] == card['uuid']]
                if not card_rulings.empty:
                    combined_rulings = "\n".join(card_rulings['text'])
                    summary = summarize(combined_rulings, 100)
                    complex_cards.append({
                        "name": card['name'],
                        "summary": summary,
                        "sentiment": "positive"
                    })
    return complex_cards
print(test_multi_udf_001(cards_df,rulings_df))

[{'name': 'Crested Sunmare', 'summary': "Crested Sunmare's ability triggers if you gained life earlier in the turn, not during the end step.", 'sentiment': 'positive'}, {'name': 'Helm of the Gods', 'summary': "You control Aura spells targeting opponents' permanents.", 'sentiment': 'positive'}]


In [ ]:
"""IMPORTANTE RIFARE UDF"""
def test_data_cleaning_001(cards: pd.DataFrame) -> List[str]:
    """Find cards with malformed quotation marks or HTML encoding."""
    malformed_cards = []
    html_entities = ["&mdash;", "&quot;", "&amp;", "&lt;", "&gt;"]
    for _, card in cards.iterrows():
        if pd.notna(card['flavorText']):
            if any(entity in card['flavorText'] for entity in html_entities):
                malformed_cards.append(card['name'])
    return malformed_cards

print(test_data_cleaning_001(cards_df))

[]


{'artist': 'creator', 'asciiName': 'alternateName', 'availability': 'availability', 'borderColor': 'color', 'cardKingdomFoilId': 'sku', 'cardKingdomId': 'sku', 'colorIdentity': 'color', 'colorIndicator': 'color', 'colors': 'color', 'convertedManaCost': 'gameItemStats', 'duelDeck': 'isPartOf', 'edhrecRank': 'rating', 'faceConvertedManaCost': 'gameItemStats', 'faceName': 'alternateName', 'flavorName': 'alternateName', 'flavorText': 'description', 'frameEffects': 'gameItemStats', 'frameVersion': 'version', 'hand': 'gameItemStats', 'hasAlternativeDeckLimit': 'isVariant', 'hasContentWarning': 'contentRating', 'hasFoil': 'isVariant', 'hasNonFoil': 'isVariant', 'isAlternative': 'isVariant', 'isFullArt': 'isVariant', 'isOnlineOnly': 'isVariant', 'isOversized': 'isVariant', 'isPromo': 'isVariant', 'isReprint': 'isVariant', 'isReserved': 'isVariant', 'isStarter': 'isVariant', 'isStorySpotlight': 'isVariant', 'isTextless': 'isVariant', 'isTimeshifted': 'isVariant', 'keywords': 'keywords', 'layout

In [16]:
pd.set_option('display.max_colwidth', None)
cards_df[["uuid","name","flavorText","text","type","power","toughness"]]

,uuid,name,flavorText,text,type,power,toughness
0,ceedc6a5-a4e3-59c4-884d-d85076d8616d,Esper Sojourners,"The wilder the frontier, the greater the need for a controlling hand.","When you cycle Esper Sojourners or it dies, you may tap or untap target permanent.\nCycling {2}{U} ({2}{U}, Discard this card: Draw a card.)",Artifact Creature — Vedalken Wizard,2.0,3.0
1,fb2e5522-233a-547f-b600-b81701a2a29e,Twilight Prophet,NaN,"Flying\nAscend (If you control ten or more permanents, you get the city's blessing for the rest of the game.)\nAt the beginning of your upkeep, if you have the city's blessing, reveal the top card of your library and put it into your hand. Each opponent los",Creature — Vampire Cleric,2.0,4.0
2,609cd717-5e92-5b4d-b04c-2e26b9965d2b,Crested Sunmare,"""It is evidence that some pure corner of the world must still exist.""\n—Djeru, former Tah-crop initiate","Other Horses you control have indestructible.\nAt the beginning of each end step, if you gained life this turn, create a 5/5 white Horse creature token.",Creature — Horse,5.0,5.0
3,4d40e2b6-9c14-574e-9096-a7974e7c03ca,Dream Stalker,What happens when it is the dream that wakes and the sleeper that fades into memory?,"When Dream Stalker enters the battlefield, return a permanent you control to its owner's hand.",Creature — Illusion,1.0,5.0
4,f5266746-23a5-56f6-8f8b-a317956203ca,Civic Wayfinder,"""These alleys are not safe. Come, I can guide you back to the market square.""","When Civic Wayfinder enters the battlefield, you may search your library for a basic land card, reveal it, and put it into your hand. If you do, shuffle your library.",Creature — Elf Druid Warrior,2.0,2.0
5,0d6962de-90fc-51ba-8246-bc794ae1ee57,Secluded Steppe,NaN,"Secluded Steppe enters the battlefield tapped.\n{T}: Add {W}.\nCycling {W} ({W}, Discard this card: Draw a card.)",Land,NaN,NaN
6,b42be0d3-320f-56c9-9464-9f6f9eb8f539,Karplusan Forest,NaN,{T}: Add {C}.\n{T}: Add {R} or {G}. Karplusan Forest deals 1 damage to you.,Land,NaN,NaN
7,16690c71-f09f-5c3a-bf28-ebdc8341be90,Diamond Mare,"When it passes, rainbows follow.","As Diamond Mare enters the battlefield, choose a color.\nWhenever you cast a spell of the chosen color, you gain 1 life.",Artifact Creature — Horse,1.0,3.0
8,fea48b09-0f75-525a-8506-034130c0f6b6,Blizzard Brawl,NaN,"Choose target creature you control and target creature you don't control. If you control three or more snow permanents, the creature you control gets +1/+0 and gains indestructible until end of turn. Then those creatures fight each other. (Each deals dama",Snow Sorcery,NaN,NaN
9,a07e27bb-cacf-5e9b-b0d3-248eac66e035,Helm of the Gods,Its blessings are as numerous as the stars of Nyx.,Equipped creature gets +1/+1 for each enchantment you control.\nEquip {1} ({1}: Attach to target creature you control. Equip only as a sorcery.),Artifact — Equipment,NaN,NaN


In [ ]:
pd.set_option('display.max_colwidth', None)
rulings_df[["uuid","text"]]

,uuid,text
0,8c8d0389-4822-5f04-bd49-b6f58def95ad,"If there are less than 7 cards in the library, look at all of them. Put two into your hand, and the rest in the graveyard."
1,8c8d0389-4822-5f04-bd49-b6f58def95ad,This is not considered a draw or a discard.
2,ceedc6a5-a4e3-59c4-884d-d85076d8616d,Cycling is an activated ability. Effects that interact with activated abilities (such as Stifle or Rings of Brighthearth) will interact with cycling. Effects that interact with spells (such as Remove Soul or Faerie Tauntings) will not.
3,ceedc6a5-a4e3-59c4-884d-d85076d8616d,"The triggered ability acts as a cycle-triggered ability or as a leaves-the-battlefield ability, as appropriate. The player who controls the triggered ability is the player who cycled the Sojourner, or the player who last controlled the Sojourner on the ba"
4,ceedc6a5-a4e3-59c4-884d-d85076d8616d,"If you cycle this card, the cycling ability goes on the stack, then the triggered ability goes on the stack on top of it. The triggered ability will resolve before you draw a card from the cycling ability."
5,ceedc6a5-a4e3-59c4-884d-d85076d8616d,"The cycling ability and the triggered ability are separate. If the triggered ability doesn’t resolve (due to being countered with Stifle, for example, or if all its targets have become illegal), the cycling ability will still resolve and you’ll draw a car"
6,ceedc6a5-a4e3-59c4-884d-d85076d8616d,You can cycle this card even if there are no targets for the triggered ability. That’s because the cycling ability itself has no targets.
7,0d6962de-90fc-51ba-8246-bc794ae1ee57,Cycling is an activated ability. Effects that interact with activated abilities (such as Stifle or Rings of Brighthearth) will interact with cycling. Effects that interact with spells (such as Remove Soul or Faerie Tauntings) will not.
8,609cd717-5e92-5b4d-b04c-2e26b9965d2b,"If a creature has been dealt damage, that damage remains marked on it until the cleanup step. If another Horse you control has been dealt lethal damage, and later in the turn Crested Sunmare leaves the battlefield, that Horse will be destroyed."
9,609cd717-5e92-5b4d-b04c-2e26b9965d2b,Crested Sunmare’s triggered ability won’t trigger unless you’ve gained life in the turn before the end step began. It can’t be satisfied by another triggered ability causing you to gain life during that end step.


In [ ]:
# Example: Sentiment Analysis with Groq
print("Running sentiment analysis on flavor text...")
cards_df['sentiment'] = cards_df['flavorText'].apply(
    lambda x: analyze_sentiment(x) if pd.notnull(x) else 0.0
)
print(cards_df[['name', 'flavorText', 'sentiment']].head())

In [ ]:
# Example: Schema.org Alignment
first_card = cards_df.iloc[0].to_dict()
print("\nSchema.org alignment for first card:")
print(align_to_schema(first_card, "Game"))

In [10]:
# Example: Entity Recognition
print("\nExtracted creature types:")
cards_df['creature_types'] = cards_df['subtypes'].apply(
    lambda x: extract_entities(x, ["CREATURE_TYPE"]) if pd.notnull(x) else []
)
print(cards_df[['name', 'creature_types']].dropna())


Extracted creature types:
                        name    creature_types
0          Ancestor's Chosen                []
1             Angel of Mercy           [Angel]
2           Aven Cloudchaser                []
3             Ballista Squad    [Human, Rebel]
4                    Bandage                []
5      Beacon of Immortality                []
6            Benalish Knight   [Human, Knight]
7            Benalish Knight   [Human, Knight]
8   Cho-Manno, Revolutionary    [Human, Rebel]
9                    Condemn                []
10         Contaminated Bond                []
11          Drudge Skeletons        [Skeleton]
12                 Demystify                []
13                      Fear                []
14             Field Marshal  [Human, Soldier]
15          Hypnotic Specter         [Specter]
16              Ghost Warden          [Spirit]
17           Glorious Anthem                []
18            Hail of Arrows                []
19            Heart of Light     

In [15]:
# Example: classification
classify_card = cards_df.iloc[0:5]
classify_card['archetype'] = classify_card['text'].apply(
    lambda x: classify_entity(x,["Aggro","Control","Combo"]) if pd.notnull(x) else "don't know"
)

print(classify_card[['name', 'text', 'archetype']])

                name                                               text  \
0  Ancestor's Chosen  First strike (This creature deals combat damag...   
1     Angel of Mercy  Flying\nWhen Angel of Mercy enters the battlef...   
2   Aven Cloudchaser  Flying (This creature can't be blocked except ...   
3     Ballista Squad  {X}{W}, {T}: Ballista Squad deals X damage to ...   
4            Bandage  Prevent the next 1 damage that would be dealt ...   

  archetype  
0     Combo  
1   Control  
2   Control  
3     Aggro  
4   Control  


C:\Users\Elia\AppData\Local\Temp\ipykernel_8900\3583654647.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  classify_card['archetype'] = classify_card['text'].apply(


In [12]:
print("\nExtracted rule:")
rulings_df['rule_name'] = rulings_df['text'].apply(
    lambda x: extract_entities(x, ["keyword_ability_name"]) if pd.notnull(x) else []
) 
print(rulings_df[["rule_name","text"]])         


Extracted rule:
                                            rule_name  \
0                                           [Bandage]   
1                                                  []   
2                             [Beacon of Immortality]   
3                                                  []   
4                                                  []   
5                                           [ability]   
6                                    [Hail of Arrows]   
7                                       [High Ground]   
8                                       [High Ground]   
9   [blocking creature, blocking creature, blockin...   
10             [enchant land, enchant green creature]   
11                                                 []   
12                                            [Clone]   
13  [enters-the-battlefield, as this creature ente...   
14                                    [Protean Hydra]   
15                          [Clone, tapped, untapped]   
16            

In [17]:
#prendo un sottoinsieme piu piccolo
cards_dfRidottissimo=cards_df.iloc[0:10]


def find_relevant_cards(ruling_text: str, top_n: int = 3) -> List[Dict]:
    """
    Find cards relevant to a ruling using multiple UDFs.
    
    1. Extracts key entities from the ruling
    2. Searches cards matching those entities
    3. Ranks by relevance using sentiment analysis
    
    Returns list of card dictionaries sorted by relevance
    """
    # Step 1: Extract key entities from ruling
    entities = extract_entities(ruling_text, ["CARD_NAME", "MECHANIC", "KEYWORD"])
    
    # Step 2: Find matching cards using entity recognition
    relevant = []
    for _, card in cards_dfRidottissimo.iterrows():
        card_entities = extract_entities(card["text"], ["MECHANIC", "KEYWORD"])
        card_entities += extract_entities(card["name"], ["CARD_NAME"])
        
        # Count matching entities
        match_score = len(set(entities) & set(card_entities))
        
        if match_score > 0:
            # Step 3: Add sentiment as tiebreaker
        #    sentiment = analyze_sentiment(card["text"])
            relevant.append({
                "card": card.to_dict(),
                "match_score": match_score,
        #        "sentiment": sentiment
            })
    
    # Sort by match score then sentiment
    #relevant.sort(key=lambda x: (-x["match_score"], -x["sentiment"]))
    relevant.sort(key=lambda x: (-x["match_score"]))
    # Return top N cards with simplified info
    return [{
        "name": item["card"]["name"],
        "match_score": item["match_score"],
        #"text": summarize(item["card"]["text"], 50),
        "type": item["card"]["type"]
    } for item in relevant[:top_n]]

# Example usage
sample_ruling = rulings_df.iloc[18]["text"]  # "You draw the card when Bandage resolves..."
relevant_cards = find_relevant_cards(sample_ruling)

print(f"Ruling: {summarize(sample_ruling, 70)}")
print("\nMost relevant cards:")
for card in relevant_cards:
    print(f"- {card['name']} ({card['match_score']} matches): {card['text']}")

Ruling: Clone's ability doesn't target.

Most relevant cards:


In [4]:
#esempio 3 query
# Join the two tables on the 'uuid' column
merged_df = pd.merge(rulings_df, cards_df[['uuid', 'subtypes']], on='uuid', how='inner')

matched = []

for _, row in merged_df.iterrows():
    subtypes = row["subtypes"]
    if pd.isna(subtypes):
        continue
    entity_list = [s.strip() for s in subtypes.split(",")]
    text = row["text"]
    
    extracted = extract_entities(text, entity_list)
    if extracted:
        matched.append(text)

print(matched)

[]


In [5]:
merged_df

,Unnamed: 0,id,date,text,uuid,subtypes
0,0,1,2007-07-15,"You draw the card when Bandage resolves, not w...",6d268c95-c176-5766-9a46-c14f739aba1c,NaN
1,1,2,2007-02-01,"If you double a negative life total, you do th...",56f4935b-f6c5-59b9-88bf-9bcce20247ce,NaN
2,2,3,2007-07-15,Beacon of Immortality’s effect counts as life ...,56f4935b-f6c5-59b9-88bf-9bcce20247ce,NaN
3,3,4,2007-07-15,"If a Beacon is countered or doesn’t resolve, i...",56f4935b-f6c5-59b9-88bf-9bcce20247ce,NaN
4,4,5,2010-08-15,The affected creature’s last known existence o...,7fef665c-36a1-5f7a-9299-cf8938708710,NaN
5,5,6,2008-08-01,The ability triggers even if the Specter’s dam...,f39b4446-0ca6-5446-8507-4b1ba37bb3e7,Specter
6,6,7,2005-06-01,You choose how the damage will be divided amon...,c19bcfc2-fc10-5040-8239-1c193098df47,NaN
